In [1]:
# Copiar código para ChatGPT

# Códigos Front
archivos_scripts = [
    "templates/header.html",
    "templates/home.html",
    "templates/progress.html",
    "templates/result.html",
    "static/styles.css",
]

def imprimir_contenido_archivos(lista_archivos):
    for archivo in lista_archivos:
        try:
            with open(archivo, 'r', encoding='utf-8') as file:
                print(f"Contenido de {archivo}:")
                print(file.read())
                print("\n" + "="*40 + "\n")
        except FileNotFoundError:
            print(f"El archivo {archivo} no se encuentra.")
        except IOError as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Llamar a la función con la lista de archivos
imprimir_contenido_archivos(archivos_scripts)

Contenido de templates/header.html:
<!DOCTYPE html>
<html lang="ca">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
    <title>{% block title %}Traductor de Documents{% endblock %}</title>
    <!-- Script per a l'scroll suau -->
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            const links = document.querySelectorAll('nav a[href^="#"]');
            links.forEach(link => {
                link.addEventListener('click', function(event) {
                    event.preventDefault();
                    document.querySelector(this.getAttribute('href')).scrollIntoView({
                        behavior: 'smooth'
                    });
                });
            });
        });
    </script>
</head>
<body>
    <header>
        <div class="container">
            <h1><a href="/">FabrIA</a></h1> <!-- Enla

In [2]:
# Copiar código para ChatGPT

# Códigos Front
archivos_scripts = [
    "app.py",
    "doc_translator.py",
]

def imprimir_contenido_archivos(lista_archivos):
    for archivo in lista_archivos:
        try:
            with open(archivo, 'r', encoding='utf-8') as file:
                print(f"Contenido de {archivo}:")
                print(file.read())
                print("\n" + "="*40 + "\n")
        except FileNotFoundError:
            print(f"El archivo {archivo} no se encuentra.")
        except IOError as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Llamar a la función con la lista de archivos
imprimir_contenido_archivos(archivos_scripts)


Contenido de app.py:
from flask import Flask, render_template, request, redirect, url_for, jsonify, send_from_directory, abort
from werkzeug.utils import secure_filename
import os
from pathlib import Path
import threading
import time
from doc_translator import traducir_doc

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = os.path.abspath('docs/uploads')
app.config['RESULT_FOLDER'] = os.path.abspath('docs/translated')
app.config['ALLOWED_EXTENSIONS'] = {'docx', 'pptx', 'pdf'}
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16 MB

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


def get_result_filename(filename):
    """
    Esta función toma el nombre del archivo original y retorna el nombre esperado del archivo traducido.
    Si el archivo es un PDF, el archivo resultante será un DOCX, de lo contrario, mantendrá la misma extensión.
    """
    file_ext = os.path.splitext(filename)[1].lower()
  